In [1]:
import numpy as np
import sys, os, h5py, re
import matplotlib.pyplot as plt
from learning_utils import prepare_data_flexdmp, calc_loss_flexdmp
from check_dyn import plot_res

In [2]:
# just for dyna error calc
from casadi_kinodynamics.utils import symbolic_robot

sym_robot = symbolic_robot.symbolic_robot(robot_name='IRB910_2r', tool_mass=0.0,
                                                tool_frame=[0., 0., 0.3, 0., 0., 0.], load_casadi_fnc=False)

In [3]:
from trajencoder.flexdmp.flexdmp import FlexDMP
from algpr.kernels import RBF
from algpr.gpr import GaussianProcessRegressor
from algpr.ppgpr import PPGPR
from glob import glob
DMP_ORDER = 20

In [4]:
Xv, Yv, Trueth, qd_trueth, qdd_trueth = prepare_data_flexdmp("../data/WithLoad/validating.hdf5", FlexDMP, DMP_ORDER=DMP_ORDER,\
                                                             return_dyddy=True, end_mass=True)

In [5]:
pred = Yv

In [6]:
error_time, error_target_q, error_dyn_loss, dyna_info = calc_loss_flexdmp(pred, Xv, Yv, Trueth, DMP_ORDER, \
                 return_dyn_loss=True, return_dyn_info=True)
print(error_time, error_target_q, error_dyn_loss)

The rate of dyna violation:  61.0 %
The peak dyna violation:  [6.52461084 0.        ]
Peak dyna violation occurs at:  [309.   0.]
0.0 0.04187942893829542 0.1252520757163481


In [18]:
training_data_root = glob("../data/WithLoad/training*[!backoff].hdf5")

In [18]:
training_data_root = glob("../data/WithLoad/training*_backoff.hdf5")

In [5]:
training_data_root = glob("../data/WithLoad/training*_backoff17.hdf5")

In [19]:
%%time
X, Y = [], []
for path in training_data_root:
    Xi, Yi, _ = prepare_data_flexdmp(path, FlexDMP, DMP_ORDER=DMP_ORDER, end_mass=True)
    X.append(Xi)
    Y.append(Yi)
X, Y = np.concatenate(X, axis=0), np.concatenate(Y, axis=0)

CPU times: user 2min 51s, sys: 1.37 s, total: 2min 53s
Wall time: 2min 54s


In [20]:
X.shape

(81536, 5)

## Full GPR with random sample

In [8]:
sample_index = np.arange(0,len(X))
sample_index = np.random.choice(sample_index, 10000, replace=False)
X_sample, Y_sample = X[sample_index], Y[sample_index]

In [9]:
gpr = GaussianProcessRegressor(kernel=RBF(l=np.array([1.8]*5), anisotropic=True))
gpr.fit(X_sample, Y_sample)

The evidence is:  -2715681988672878.0


-2715681988672878.0

In [10]:
%%time
for _ in range(10):
    pred = gpr.predict(Xv)

CPU times: user 3.09 s, sys: 7.09 s, total: 10.2 s
Wall time: 1.27 s


In [11]:
error_time, error_target_q, error_dyn_loss, dyna_info = calc_loss_flexdmp(pred, Xv, Yv, Trueth, DMP_ORDER, \
                 return_dyn_loss=True, return_dyn_info=True)
print(error_time, error_target_q, error_dyn_loss)

The rate of dyna violation:  12.5 %
The peak dyna violation:  [74.80643658  0.        ]
Peak dyna violation occurs at:  [388.   0.]
0.019569999999999987 0.09793178306172443 0.16557152968554878


In [13]:
# suboptimality
suboptimality = (pred[:,-1] - Yv[:,-1]).sum() / Yv[:,-1].sum()
# print('Subopt: {:.2f}'.format(suboptimality*100))
suboptimality * 100

2.508653984085708

The rate of dyna violation:  57.49999999999999 %  
The peak dyna violation:  [143.86973349   3.95647664]  
Peak dyna violation occurs at:  [111. 391.]  
0.009920000000000007 0.05756379409805197 0.5281403209469206

## PP-GPR

In [14]:
ppgpr = PPGPR(kernel=RBF(l=np.array([1.8]*5), anisotropic=True))
ppgpr.fit(X, Y, m=5000)

In [15]:
%%time
for _ in range(10):
    pred = ppgpr.predict(Xv)

CPU times: user 1.8 s, sys: 4.53 s, total: 6.33 s
Wall time: 600 ms


In [16]:
error_time, error_target_q, error_dyn_loss, dyna_info = calc_loss_flexdmp(pred, Xv, Yv, Trueth, DMP_ORDER, \
                 return_dyn_loss=True, return_dyn_info=True)
print(error_time, error_target_q, error_dyn_loss)

The rate of dyna violation:  0.0 %
The peak dyna violation:  [0. 0.]
Peak dyna violation occurs at:  [0. 0.]
0.01607 0.09162078382018418 0.0


In [17]:
# suboptimality
suboptimality = (pred[:,-1] - Yv[:,-1]).sum() / Yv[:,-1].sum()
# print('Subopt: {:.2f}'.format(suboptimality*100))
suboptimality * 100

2.3298947903442824

The rate of dyna violation:  60.0 %  
The peak dyna violation:  [12.13908725  0.        ]  
Peak dyna violation occurs at:  [126.   0.]  
0.0031500000000000026 0.044353611229823516 0.20313834457090763

In [65]:
for index in range(0, 400):
    plot_res(sym_robot.q_max, sym_robot.q_min, sym_robot.qd_max, sym_robot.qdd_max,\
                 index, pred, Xv, Yv, Trueth, qd_trueth, qdd_trueth, dmp3=FlexDMP, DMP_ORDER=DMP_ORDER)


KeyboardInterrupt



Error in callback <function flush_figures at 0x7f28cdf63d30> (for post_execute):



KeyboardInterrupt

